In [1]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool
from getpass import getpass
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import geocoder
import ast

api_key = getpass("Enter your OpenAI API key: ")

In [ ]:
df_crash = pd.read_csv("processed_data/crash_data.csv")
df_acceleration = pd.read_csv("processed_data/harsh_accel_data.csv")
df_braking = pd.read_csv("processed_data/harsh_braking_data.csv")
df_crash.head()

(290887, 13)

In [3]:
df = pd.read_csv("processed_data/crash_data.csv")

In [19]:
from typing import List, Dict

# Tool: Calculate Distance
@tool
def calculate_distance(lat1: float, lon1: float) -> List[Dict]:
    """
    Calculate the distance in miles between a given point (lat1, lon1) and 
    each row's geographic point using the Haversine formula. Add a `distance` column.
    """
    # if isinstance(df, str):
    #     df = pd.DataFrame(ast.literal_eval(df))  # Safely parse string to DataFrame
    # elif isinstance(df, dict):
    #     df = pd.DataFrame(df)
    df = pd.read_csv("processed_data/crash_data.csv")

    def haversine(row):
        lat2, lon2 = row['LATITUDE'], row['LONGITUDE']
        lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(np.radians, [lat1, lon1, lat2, lon2])
        dlat = lat2_rad - lat1_rad
        dlon = lon2_rad - lon1_rad
        a = np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        r = 3963  # Radius of Earth in miles
        return r * c

    df['distance'] = df.apply(haversine, axis=1)
    df.to_csv("processed_data/temp.csv")
    return df

# Tool: Get Geographic Coordinates
@tool
def get_geographic_coordinates(placename: str) -> tuple:
    """
    Fetch geographic coordinates (latitude and longitude) for a given place name.
    """
    try:
        search = geocoder.geonames(placename, maxRows=1, country='US', adminCode1='CA', adminCode2='037', key='dongim04')
        latitude = search.lat
        longitude = search.lng
        if latitude is None or longitude is None:
            raise ValueError("Coordinates not found.")
        return latitude, longitude
    except Exception as e:
        print(f"Error fetching coordinates for {placename}: {e}")
        return None, None

# Tool: Filter DataFrame
@tool
def filter_df(conditions: dict) -> List[Dict]:
    """
    Filter a DataFrame based on multiple conditions from user query provided as a dictionary.
    example parameter: conditions = {'year': 2020, 'poi_type': 'School'}
    """
    # if isinstance(df, str):
    #     df = pd.DataFrame(ast.literal_eval(df))  # Safely parse string to DataFrame
    # elif isinstance(df, dict):
    #     df = pd.DataFrame(df)
    df = pd.read_csv("processed_data/crash_data.csv")

    filtered_df = df.copy()
    for column, value in conditions.items():
        if column in filtered_df.columns:
            filtered_df = filtered_df[filtered_df[column] == value]
        else:
            raise ValueError(f"Column '{column}' does not exist in the DataFrame.")
    return filtered_df

# CSV Agent Setup
csv_agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-4o-mini", api_key=api_key),
    path="processed_data/crash_data.csv",
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
    extra_tools=[calculate_distance, get_geographic_coordinates],
    prefix=(
        "First filter out necessary rows using appropriate column values (e.g., year, poi_type, etc.). "
        "Based on the user query, come up with conditions dictionary. For example, conditions = {'year': 2020, 'poi_type': 'School'}"
        "If necessary, use the `calculate_distance` function to calculate distances between locations. "
        "Then, use the entire 290887 distance values for calculation afterward. NEVER use sample data. Read csv data from 'processed_data/temp.csv'"
    )
)

Info for the nearest crash to UCLA
How many crashes happened in 2020 within 20 miles from Hollywood?

In [20]:
query = "How many crashes happened in 2020 within 20 miles from Hollywood?"
response = csv_agent.invoke(query)
print("Question:", response.get("input"))
print("Response:", response.get("output"))



> Entering new AgentExecutor chain...

Invoking: `get_geographic_coordinates` with `{'placename': 'Hollywood'}`


('34.09834', '-118.32674')
Invoking: `calculate_distance` with `{'lat1': 34.09834, 'lon1': -118.32674}`


         LATITUDE   LONGITUDE  \
0       34.022659 -118.438652   
1       33.771053 -118.240303   
2       34.025581 -118.335114   
3       34.164879 -118.410530   
4       33.969830 -118.347229   
...           ...         ...   
290882  34.141766 -118.374763   
290883  34.281681 -118.294502   
290884  34.219341 -118.398750   
290885  34.228191 -118.599525   
290886  34.228191 -118.599525   

                                                   ARC_ID    YEAR   VH  CYC  \
0       3815613985340226038495917846_38156133053402269...  2018.0  0.0  1.0   
1       3817593925337726257993907375_38175966053377030...  2018.0  0.0  1.0   
2       38166473753402558910272263152_3816648955340255...  2018.0  0.0  0.0   
3       3815895015341648791916620220_38158843353416488...  2019.0